In [9]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [4]:
# Wine Enthusiasts wine reviews website WineMag.com.  Scraping the results of wines from the USA reviewed 
# in February 2019 sorted by review date, most recent first

url='https://www.winemag.com/?s=&drink_type=wine&country=US&pub_date_web=2019,2019-02-01&page=1&sort_by=pub_date_web&sort_dir=desc&search_type=reviews'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Grab a list of links for each wine's review page from the results page of WineMag reviews

wines_url_list=[]

wines_ls = soup.find_all('a', class_="review-listing row" )

for link in wines_ls: 
    href=link['href']
    wines_url_list.append(href.strip())

wines_url_list

['https://www.winemag.com/buying-guide/lucia-2017-garys-vineyard-pinot-noir-santa-lucia-highlands/',
 'https://www.winemag.com/buying-guide/laetitia-2016-la-coupelle-pinot-noir-arroyo-grande-valley/',
 'https://www.winemag.com/buying-guide/lucia-2017-soberanes-vineyard-pinot-noir-santa-lucia-highlands/',
 'https://www.winemag.com/buying-guide/gary-farrell-2016-rochioli-allen-vineyards-pinot-noir-russian-river-valley/',
 'https://www.winemag.com/buying-guide/gary-farrell-2016-lancel-creek-vineyard-pinot-noir-russian-river-valley/',
 'https://www.winemag.com/buying-guide/laetitia-2016-les-galets-pinot-noir-arroyo-grande-valley/',
 'https://www.winemag.com/buying-guide/carlisle-2016-piner-olivet-ranches-zinfandel-russian-river-valley/',
 'https://www.winemag.com/buying-guide/cobb-2016-diane-cobb-coastlands-vineyard-pinot-noir-sonoma-coast/',
 'https://www.winemag.com/buying-guide/styring-2015-wit-reserve-estate-pinot-noir-ribbon-ridge/',
 'https://www.winemag.com/buying-guide/freeman-2016

In [6]:


wine_list=[]

columns = ['title', 'wine', 'vintage', 'vinyard', 'variety', 'avi', 'region', 'state',
           'alcohol_content', 'size', 'winetype', 'price', 'score','dt_published', 'taster']

wines_df = pd.DataFrame(wine_list, columns = columns)
index=0

# Follow each link and scrape the wine's review

for link in wines_url_list:
    browser.visit(link)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    wine_elements=[]

    title=soup.find('div', class_="article-title" ).text.split('[')[0]
    score=soup.find('div', class_='rating').text.split()[0]

    primary_info=soup.find_all('li', class_='row')
    for row in primary_info: 
        if row.text.strip()!= '\n':
            wine_elements.append(row.text.split('\n')[5])

    prices=wine_elements[0].split(',')
    price=prices[0]
    vinyard=wine_elements[1]
    variety=wine_elements[2]
    avi=wine_elements[3].split(',')[0]
    region=wine_elements[3].split(',')[1]
    state=wine_elements[3].split(',')[2]
    wine=wine_elements[4]
    alcohol=wine_elements[5]
    size=wine_elements[6]
    winetype=wine_elements[7]
    dt_published=wine_elements[8]
    taster=soup.find('div', class_='name').text
    vintage = re.findall('\d\d\d\d', title)[0]

    wines_df.set_value(index, "title", title)
    wines_df.set_value(index, "wine", wine)
    wines_df.set_value(index, "vintage", vintage)    
    wines_df.set_value(index, "vinyard", vinyard)
    wines_df.set_value(index, "variety", variety)
    wines_df.set_value(index, "avi", avi)
    wines_df.set_value(index, "region", region)
    wines_df.set_value(index, "state", state)
    wines_df.set_value(index, "alcohol_content", alcohol)
    wines_df.set_value(index, "size", size)
    wines_df.set_value(index, "winetype", winetype)
    wines_df.set_value(index, "score", score)    
    wines_df.set_value(index, "price", price)
    wines_df.set_value(index, "dt_published", dt_published)
    wines_df.set_value(index, "taster", taster)

    index += 1
    

wines_df.head(10)

/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:44: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:45: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:46: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[

,title,wine,vintage,vinyard,variety,avi,region,state,alcohol_content,size,winetype,price,score,dt_published,taster
0,Lucia 2017 Garys' Vineyard Pinot Noir (Santa L...,Lucia,2017,Garys' Vineyard,Pinot Noir,Santa Lucia Highlands,Central Coast,California,14.2%,750 ml,Red,$65,97,6/1/2019,Matt Kettmann
1,Laetitia 2016 La Coupelle Pinot Noir (Arroyo G...,Laetitia,2016,La Coupelle,Pinot Noir,Arroyo Grande Valley,Central Coast,California,14.3%,750 ml,Red,$62,96,6/1/2019,Matt Kettmann
2,Lucia 2017 Soberanes Vineyard Pinot Noir (Sant...,Lucia,2017,Soberanes Vineyard,Pinot Noir,Santa Lucia Highlands,Central Coast,California,14.3%,750 ml,Red,$65,96,6/1/2019,Matt Kettmann
3,Gary Farrell 2016 Rochioli-Allen Vineyards Pin...,Gary Farrell,2016,Rochioli-Allen Vineyards,Pinot Noir,Russian River Valley,Sonoma,California,14.1%,750 ml,Red,$80,96,6/1/2019,Virginie Boone
4,Gary Farrell 2016 Lancel Creek Vineyard Pinot ...,Gary Farrell,2016,Lancel Creek Vineyard,Pinot Noir,Russian River Valley,Sonoma,California,13.9%,750 ml,Red,$60,96,6/1/2019,Virginie Boone
5,Laetitia 2016 Les Galets Pinot Noir (Arroyo Gr...,Laetitia,2016,Les Galets,Pinot Noir,Arroyo Grande Valley,Central Coast,California,14.2%,750 ml,Red,$62,96,6/1/2019,Matt Kettmann
6,Carlisle 2016 Piner-Olivet Ranches Zinfandel (...,Carlisle,2016,Piner-Olivet Ranches,Zinfandel,Russian River Valley,Sonoma,California,15%,750 ml,Red,$40,96,6/1/2019,Virginie Boone
7,Cobb 2016 Diane Cobb Coastlands Vineyard Pinot...,Cobb,2016,Diane Cobb Coastlands Vineyard,Pinot Noir,Sonoma Coast,Sonoma,California,13.6%,750 ml,Red,$120,96,6/1/2019,Virginie Boone
8,Styring 2015 Wit Reserve Estate Pinot Noir (Ri...,Styring,2015,Wit Reserve Estate,Pinot Noir,Ribbon Ridge,Willamette Valley,Oregon,15%,750 ml,Red,$65,96,6/1/2019,Paul Gregutt
9,Freeman 2016 Yu-Ki Estate Pinot Noir (Sonoma C...,Freeman,2016,Yu-Ki Estate,Pinot Noir,Sonoma Coast,Sonoma,California,14.1%,750 ml,Red,$68,95,6/1/2019,Jim Gordon


In [10]:
# Save wines to csv file
wines_file_path = os.path.join("Resources","winesUSA.csv")
wines_df.to_csv(wines_file_path)